In [1]:
%load_ext autoreload
%autoreload 2

from time import time
import pandas as pd
import numpy as np
import os
from collections import Counter, defaultdict

In [2]:
import pyterrier as pt
pt.init()

PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [ ]:
from jnius import autoclass
tokeniser = autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

In [ ]:
def terrier_tokenizer(raw_utterance):
    new_utterance = " ".join(tokeniser.getTokens(raw_utterance))
    return new_utterance

# Retrieve docs per MANUAL query

In [5]:
index_ref = pt.IndexRef.of("/data3/muntean/conversational-cache/indexes/CAST2020-stemmed/data.properties")
index = pt.IndexFactory.of(index_ref)

di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()

print(index.getCollectionStatistics())

10:32:13.282 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 294.8 MiB of memory would be required.
10:32:13.314 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 2 GiB of memory would be required.
Number of documents: 38636520
Number of terms: 9333281
Number of postings: 1052145306
Number of fields: 0
Number of tokens: 1300558544
Field names: []
Positions:   false



In [6]:
qrel_path = "../data/CAST_qrels/qrels-docs.2019.txt"
qrels_df = pd.read_csv(qrel_path, delimiter=" ", header=None)
qrels_df[[3]] = qrels_df[[3]].astype(int)
qrels_df = qrels_df.drop([1], axis=1)
qrels_df.columns=["qid", "docno", "label"]
qrels = qrels_df

In [7]:
topics_path='../data/CAST-2019/test_manual_utterance.tsv' #manual

topics_df = pd.read_csv(topics_path, delimiter="\t", header=None)

topics_df[2] = topics_df[1].apply(lambda s: terrier_tokenizer(s))

topics_df = topics_df.drop([1], axis=1)
topics_df.columns=["qid", "query"]

topics = topics_df
topics.head()

,qid,query
0,31_1,what is throat cancer
1,31_2,is throat cancer treatable
2,31_3,tell me about lung cancer
3,31_4,what are lung cancer s symptoms
4,31_5,can lung cancer spread to the throat


In [8]:
%%time
DPH = pt.BatchRetrieve(index, wmodel="DPH", num_results=1000)  
# res = DPH.transform(topics)

CPU times: user 30.3 ms, sys: 769 µs, total: 31.1 ms
Wall time: 234 ms


In [9]:
%%time
pt.Experiment([DPH], topics, qrels, names=["DPH"], 
              eval_metrics=["map", "recip_rank", "recall_200", "P_3", "P_1", "ndcg_cut_3"])

CPU times: user 2min 25s, sys: 17.3 s, total: 2min 42s
Wall time: 1h 10min 6s


,name,map,recip_rank,recall_200,P_3,P_1,ndcg_cut_3
0,DPH,0.296065,0.672303,0.596395,0.560694,0.543353,0.397444


In [10]:
%%time
res_per_query = pt.Experiment([DPH], topics, qrels, names=["DPH"], 
              eval_metrics=["map", "recip_rank", "recall_200", "P_3", "P_1", "ndcg_cut_3"], perquery=True)

CPU times: user 1min 42s, sys: 4.58 s, total: 1min 47s
Wall time: 8min 23s


/data3/muntean/conversational-cache/conversational-cache/lib/python3.9/site-packages/pyterrier/pipelines.py:110: UserWarning: 306 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [13]:
res_per_query.head(50)

,name,qid,measure,value
0,DPH,31_1,map,0.351272
1,DPH,31_1,recip_rank,1.000000
2,DPH,31_1,P_1,1.000000
3,DPH,31_1,P_3,1.000000
4,DPH,31_1,recall_200,0.629213
5,DPH,31_1,ndcg_cut_3,0.824020
6,DPH,31_2,map,0.086993
7,DPH,31_2,recip_rank,1.000000
8,DPH,31_2,P_1,1.000000
9,DPH,31_2,P_3,0.333333
